# Joern-ey into ILP

In [3]:
import pandas as pd

In [33]:
ilp_data = pd.read_csv("../data/ilp_dataset.csv.gz")
ilp_data = ilp_data.drop('Unnamed: 0', axis='columns')

In [34]:
ilp_data

,testcase_ID,filename,code,flaw,flaw_loc,bug,code_length
0,62804,000/062/804/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,False,1722
1,62852,000/062/852/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,30,False,1674
2,62869,000/062/869/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,32,False,1760
3,62900,000/062/900/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,30,False,1680
4,232012,000/232/012/CWE122_Heap_Based_Buffer_Overflow_...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-122,31,False,1723
5,-62804,000/062/804/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,33,True,1618
6,-62852,000/062/852/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,30,True,1570
7,-62869,000/062/869/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,32,True,1616
8,-62900,000/062/900/CWE121_Stack_Based_Buffer_Overflow...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-121,30,True,1576
9,-232012,000/232/012/CWE122_Heap_Based_Buffer_Overflow_...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,CWE-122,31,True,1619


Now do it for all our examples:

In [4]:
import os
import subprocess
import tempfile

In [39]:
testcase_IDs = []
flaws = []
bugs = []
code_lengths = []
trees = []

In [40]:
def generate_prolog(testcase):
    tmp_dir = tempfile.TemporaryDirectory()

    for file in testcase.itertuples():
        short_filename = file.filename.split("/")[-1]
        with open(tmp_dir.name + "/" + short_filename, 'w') as f:
            f.write(file.code)

    subprocess.check_call(["/joern/joern-parse", "--out", tmp_dir.name + "/cpg.bin.zip", tmp_dir.name])

    tree = subprocess.check_output(
        "cd /joern && /joern/joern-query --cpg "+tmp_dir.name + "/cpg.bin.zip -f /project/code/joern_cfg_to_prolog.scala",
        shell=True,
        universal_newlines=True,
    )

    testcase_IDs.append(file.testcase_ID)
    flaws.append( file.flaw)
    bugs.append(file.bug)
    code_lengths.append(file.code_length)
    trees.append(tree)

    tmp_dir.cleanup()

In [ ]:
ilp_data.groupby('testcase_ID').apply(generate_prolog)

In [29]:
prolog = pd.DataFrame({
    'testcase_ID': testcase_IDs,
    'flaw': flaws,
    'bug': bugs,
    'code_length': code_lengths,
    'tree': trees,
})
prolog = prolog[1:10]  # when we run apply it duplicates the first group
prolog

,testcase_ID,flaw,bug,code_length,tree
0,-232012,CWE-122,True,1619,"% START: Generated Prolog\n% AST\nast(67, 64)...."
1,-232012,CWE-122,True,1619,"% START: Generated Prolog\n% AST\nast(46, 47)...."
2,-62900,CWE-121,True,1576,% START: Generated Prolog\n% AST\nast(106_memm...
3,-62869,CWE-121,True,1616,"% START: Generated Prolog\n% AST\nast(60, 50)...."
4,-62852,CWE-121,True,1570,"% START: Generated Prolog\n% AST\nast(67, 47)...."
5,-62804,CWE-121,True,1618,"% START: Generated Prolog\n% AST\nast(58, 57)...."
6,62804,CWE-121,False,1722,"% START: Generated Prolog\n% AST\nast(23, 33)...."
7,62852,CWE-121,False,1674,"% START: Generated Prolog\n% AST\nast(54, 31)...."
8,62869,CWE-121,False,1760,"% START: Generated Prolog\n% AST\nast(32, 54)...."
9,62900,CWE-121,False,1680,"% START: Generated Prolog\n% AST\nast(52, 23)...."


In [45]:
prolog.to_csv("../data/ilp_prolog_data.csv.gz")

In [5]:
prolog = pd.read_csv("../data/ilp_prolog_data.csv.gz")

In [6]:
print(prolog['tree'][0])

% START: Generated Prolog
% AST
ast(46, 47).
 ast(79, 47).
 ast(87, 47).
 ast(65, 47).
 ast(30, 26).
 ast(86, 26).
 ast(71, 26).
 ast(76, 26).
 ast(73, 56).
 ast(25, 56).
 ast(38, 56).
 ast(60, 90).
 ast(83, 90).
 ast(0, 90).
 ast(42, 90).
 ast(84, 85).
 ast(54, 85).
 ast(41, 85).
 ast(51, 29).
 ast(75, 29).
 ast(80, 29).
 ast(50, 66).
 ast(31, 66).
 ast(63, 66).
 ast(37, 66).
 ast(33, 66).
 ast(61, 89).
 ast(45, 89).
 ast(55, 89).
 ast(74, 89).
 ast(68, 58).
 ast(88, 58).
 ast(64, 58).
 ast(39, 62).
 ast(82, 62).
 ast(67, 62).
 ast(69, 36).
 ast(72, 36).
 ast(48, 36).
 ast(28, 36).
 ast(70, 40).
 ast(27, 40).
 ast(34, 40).
 ast(49, 91).
 ast(77, 91).
 ast(44, 91).
 ast(59, 32).
 ast(53, 32).
 ast(35, 32).
 ast(52, 57).
 ast(78, 57).
 ast(81, 57).
 ast(18, 121).
 ast(24, 121).
 ast(5, 47).
 ast(10, 57).
 ast(1, 29).
 ast(23, 26).
 ast(11, 89).
 ast(12, 26).
 ast(14, 62).
 ast(6, 47).
 ast(8, 58).
 ast(17, 66).
 ast(3, 89).
 ast(9, 36).
 ast(7, 91).
 ast(20, 90).
 ast(19, 90).
 ast(13, 

In [7]:
print(prolog['tree'][1])

% START: Generated Prolog
% AST
ast(106_memmove_01_c_30_26, 109_memmove_01_c_30_21).
 ast(107_memmove_01_c_30_21, 109_memmove_01_c_30_21).
 ast(109_memmove_01_c_30_21, 111_memmove_01_c_30_8).
 ast(111_memmove_01_c_30_8, 125_memmove_01_c_27_8).
 ast(112_memmove_01_c_29_40, 114_memmove_01_c_29_33).
 ast(114_memmove_01_c_29_33, 117_memmove_01_c_29_30).
 ast(115_memmove_01_c_29_30, 117_memmove_01_c_29_30).
 ast(117_memmove_01_c_29_30, 121_memmove_01_c_29_8).
 ast(118_memmove_01_c_29_22, 121_memmove_01_c_29_8).
 ast(119_memmove_01_c_29_16, 121_memmove_01_c_29_8).
 ast(121_memmove_01_c_29_8, 125_memmove_01_c_27_8).
 ast(122_memmove_01_c_27_12, 123_memmove_01_c_27_12).
 ast(123_memmove_01_c_27_12, 125_memmove_01_c_27_8).
 ast(124, 125_memmove_01_c_27_8).
 ast(125_memmove_01_c_27_8, 138_memmove_01_c_22_0).
 ast(126_memmove_01_c_25_25, 128_memmove_01_c_25_18).
 ast(128_memmove_01_c_25_18, 130_memmove_01_c_25_11).
 ast(129_memmove_01_c_25_12, 130_memmove_01_c_25_11).
 ast(130_memmove_01_c_25_11,

In [8]:
import re

In [10]:
def fixprolog(testcase):
    find_node_ids = re.compile('\((\w+), (\w+)\)')
    replacement_node_ids = '({bug}_{testcase_id}_\\1, {bug}_{testcase_id}_\\2)'.format(
        bug='bad' if testcase.bug else 'good',
        testcase_id=abs(testcase.testcase_ID),
    )
    return find_node_ids.sub(replacement_node_ids, testcase['tree'])

In [11]:
list(map(fixprolog,prolog['tree']))

AttributeError: 'str' object has no attribute 'bug'

In [12]:
prolog

,Unnamed: 0,testcase_ID,flaw,bug,code_length,tree
0,1,-232012,CWE-122,True,1619,"% START: Generated Prolog\n% AST\nast(46, 47)...."
1,2,-62900,CWE-121,True,1576,% START: Generated Prolog\n% AST\nast(106_memm...
2,3,-62869,CWE-121,True,1616,"% START: Generated Prolog\n% AST\nast(60, 50)...."
3,4,-62852,CWE-121,True,1570,"% START: Generated Prolog\n% AST\nast(67, 47)...."
4,5,-62804,CWE-121,True,1618,"% START: Generated Prolog\n% AST\nast(58, 57)...."
5,6,62804,CWE-121,False,1722,"% START: Generated Prolog\n% AST\nast(23, 33)...."
6,7,62852,CWE-121,False,1674,"% START: Generated Prolog\n% AST\nast(54, 31)...."
7,8,62869,CWE-121,False,1760,"% START: Generated Prolog\n% AST\nast(32, 54)...."
8,9,62900,CWE-121,False,1680,"% START: Generated Prolog\n% AST\nast(52, 23)...."


In [13]:
prolog['tree'] = prolog.apply(fixprolog, axis='columns')

In [16]:
print(prolog['tree'][0])

% START: Generated Prolog
% AST
ast(bad_232012_46, bad_232012_47).
 ast(bad_232012_79, bad_232012_47).
 ast(bad_232012_87, bad_232012_47).
 ast(bad_232012_65, bad_232012_47).
 ast(bad_232012_30, bad_232012_26).
 ast(bad_232012_86, bad_232012_26).
 ast(bad_232012_71, bad_232012_26).
 ast(bad_232012_76, bad_232012_26).
 ast(bad_232012_73, bad_232012_56).
 ast(bad_232012_25, bad_232012_56).
 ast(bad_232012_38, bad_232012_56).
 ast(bad_232012_60, bad_232012_90).
 ast(bad_232012_83, bad_232012_90).
 ast(bad_232012_0, bad_232012_90).
 ast(bad_232012_42, bad_232012_90).
 ast(bad_232012_84, bad_232012_85).
 ast(bad_232012_54, bad_232012_85).
 ast(bad_232012_41, bad_232012_85).
 ast(bad_232012_51, bad_232012_29).
 ast(bad_232012_75, bad_232012_29).
 ast(bad_232012_80, bad_232012_29).
 ast(bad_232012_50, bad_232012_66).
 ast(bad_232012_31, bad_232012_66).
 ast(bad_232012_63, bad_232012_66).
 ast(bad_232012_37, bad_232012_66).
 ast(bad_232012_33, bad_232012_66).
 ast(bad_232012_61, bad_232012_89)

In [17]:
prolog

,Unnamed: 0,testcase_ID,flaw,bug,code_length,tree
0,1,-232012,CWE-122,True,1619,% START: Generated Prolog\n% AST\nast(bad_2320...
1,2,-62900,CWE-121,True,1576,% START: Generated Prolog\n% AST\nast(bad_6290...
2,3,-62869,CWE-121,True,1616,% START: Generated Prolog\n% AST\nast(bad_6286...
3,4,-62852,CWE-121,True,1570,% START: Generated Prolog\n% AST\nast(bad_6285...
4,5,-62804,CWE-121,True,1618,% START: Generated Prolog\n% AST\nast(bad_6280...
5,6,62804,CWE-121,False,1722,% START: Generated Prolog\n% AST\nast(good_628...
6,7,62852,CWE-121,False,1674,% START: Generated Prolog\n% AST\nast(good_628...
7,8,62869,CWE-121,False,1760,% START: Generated Prolog\n% AST\nast(good_628...
8,9,62900,CWE-121,False,1680,% START: Generated Prolog\n% AST\nast(good_629...


In [23]:
good_example = prolog[prolog['testcase_ID'] == 62852].iloc[0]
bad_example = prolog[prolog['testcase_ID'] == -62852].iloc[0]

In [49]:
def extract_node_ids(tree):
    nodes = set()
    
    for line in tree.split('\n'):
        line = line.strip()
        if not line.startswith("%") and line:
            parent, child = line[4:-2].split(", ")
            nodes.add(parent)
            nodes.add(child)
            
    return nodes

In [50]:
good_nodes = extract_node_ids(good_example.tree)
bad_nodes = extract_node_ids(bad_example.tree)

In [53]:
positive_examples = [
    'bug('+node_id+')' for node_id in bad_nodes
]

negative_examples = [
    'bug('+node_id+')' for node_id in good_nodes
]

base_rules = [
]

meta_rules = [
    'metarule([P,Q],[P,A,B],[[Q,A,B]]).',
    'metarule([P,Q,R],[P,A,B],[[Q,A,B],[R,A,B]]).',
    'metarule([P,Q,R],[P,A,B],[[Q,A,C],[R,C,B]]).',
]



In [54]:
script_template = """
:- use_module('metagol').

%% metagol settings
{settings}

%% background knowledge
{background_knowledge}

%% metarules
{metarules}

%% learning task
:-
  %% positive examples
  Pos = [
    {positive_examples}
  ],
  %% negative examples
  Neg = [
    {negative_examples}
  ],
  learn(Pos,Neg).
"""

script = script_template.format(
    background_knowledge='\n'.join([good_example.tree, bad_example.tree] + base_rules),
    settings="""
    body_pred(ast/2).
    body_pred(cfg/2).
    """,
    metarules='\n'.join(meta_rules),
    positive_examples=',\n'.join(positive_examples),
    negative_examples=',\n'.join(negative_examples),
)
print(script)


:- use_module('metagol').

%% metagol settings

    body_pred(ast/2).
    body_pred(cfg/2).
    

%% background knowledge
% START: Generated Prolog
% AST
ast(good_62852_54, good_62852_31).
 ast(good_62852_67, good_62852_31).
 ast(good_62852_52, good_62852_31).
 ast(good_62852_48, good_62852_31).
 ast(good_62852_65, good_62852_56).
 ast(good_62852_53, good_62852_56).
 ast(good_62852_50, good_62852_56).
 ast(good_62852_60, good_62852_57).
 ast(good_62852_38, good_62852_57).
 ast(good_62852_22, good_62852_57).
 ast(good_62852_29, good_62852_57).
 ast(good_62852_19, good_62852_40).
 ast(good_62852_27, good_62852_40).
 ast(good_62852_32, good_62852_40).
 ast(good_62852_62, good_62852_28).
 ast(good_62852_37, good_62852_28).
 ast(good_62852_68, good_62852_28).
 ast(good_62852_49, good_62852_28).
 ast(good_62852_43, good_62852_28).
 ast(good_62852_26, good_62852_64).
 ast(good_62852_41, good_62852_64).
 ast(good_62852_63, good_62852_64).
 ast(good_62852_47, good_62852_23).
 ast(good_62852_25

In [38]:
with open("test.pl", 'w') as f:
    f.write(script)

In [35]:
ls

'Joern-ey into ILP.ipynb'
 Untitled.ipynb
 Untitled1.ipynb
 __init__.py
 __pycache__/
 baseline-model-binary.ipynb
 baseline-model-comparison.ipynb
 baseline_model_final.ipynb
 bug_picking.ipynb
 experimenting-with-python-clang-bindings.ipynb
 exploring_juliet.ipynb
 exploring_vdisc.ipynb
 generate_minimal_ilp_dataset.ipynb
 joern-example.ipynb
 joern_cfg_to_prolog.scala
 json2tree.ipynb
 preprocess_code.py
 run_graph2vec_on_juliet_dataset.ipynb
 runmetagol-example.pl
 test.pl
 test.scala
 testsuite-multilabel.py
 try_exploringvdisc.ipynb
 visualisation.m
